## User signals to boost relevance based suggestions

For relevance based suggestions we agreed on having all the user signals from a dataset and use them to build a separate index for our suggestions.

This dataset contains some time-based-data for users interactions on an ecommerce, in a funnel-based manner. Those signals are:

- Search: A user types a query in the search box and sees the results
- Click: The user, coming from a specific search, clicks a product that could be relevant to their search
- Add-to-cart: The user adds their product to the cart
- Purchase: The user eventually transacts and purchase

Below we'll see the distribution of each of these events

In [77]:
import pandas as pd

signals_df = pd.read_csv("signals.csv")

signals_df.head(30)

/tmp/ipykernel_29231/2332084007.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  signals_df = pd.read_csv("signals.csv")


,query_id,user,type,target,signal_time
0,u2_0_1,u2,query,nook,2019-07-31 08:49:07.3116
1,u2_1_2,u2,query,rca,2020-05-04 08:28:21.1848
2,u3_0_1,u3,query,macbook,2019-12-22 00:07:07.0152
3,u4_0_1,u4,query,Tv antenna,2019-08-22 23:45:54.1030
4,u5_0_1,u5,query,AC power cord,2019-10-20 08:27:00.1600
5,u6_0_1,u6,query,Watch The Throne,2019-09-18 11:59:53.7470
6,u7_0_1,u7,query,Camcorder,2020-02-25 13:02:29.3089
7,u9_0_1,u9,query,wireless headphones,2020-04-26 04:26:09.7198
8,u10_0_1,u10,query,Xbox,2019-09-13 16:26:12.0132
9,u10_0_2,u10,query,Xbox360,2019-09-13 17:41:30.7109


In [76]:
grouped = signals_df.groupby("type").count().reset_index
grouped

<bound method DataFrame.reset_index of              query_id    user  target  signal_time
type                                              
add-to-cart    404454  404454  404454       404454
click          647684  647684  647684       647684
purchase       395008  395008  395008       395008
query          725459  725459  725459       725459>

### Practical use case
For our example, we'll only focus on queries, which are the textual signals that we'll build the suggestions from.
We'll get all the queries done on our dataset.

In [14]:
query_df = signals_df[signals_df.type=="query"]

queries = query_df['target'].to_list()

len(queries)    

725459

Almost a million searches, which could be quite a bit for our purposes. What if we aggregate them by "target" (query text) and see the results?

In [15]:
agg_df = signals_df[signals_df.type=="query"].groupby('target').count().reset_index()
agg_df.sort_values('query_id', ascending=False)

agg_df[['target','query_id']].head(20) # ~35K rows

,target,query_id
0,#,185
1,*,178
2,007,7
3,1 TB,1
4,1 TB external hard drive,1
5,1 tb,11
6,1 tb External hard drive,1
7,1 tb external hard drive,7
8,1 tb hard drive,11
9,10 Kicker,1


This aggregation allows us to do an insightful analysis: infrequent queries and their results vs frequent queries and their results. It might be more relevant to tune the query "10 subwoofer" which has been searched 25 times in our dataset,than "1 TB external hard drive" which has been searched only once.

---

But for the most part, we care more about our search results. We don't want to suggest anything that yields 0 results, so the most important sanity check is to run all the candidate user searches again against our search engine to see if they are still user-relevant.

For this, we'll follow the following approach:

- Discard automatically all suggestions with _less than two characters_
- Run the candidate search on our system, and **if** the search yields 10 or more results
- We store this candidate as a pontential suggestion

In [39]:
import requests

def has_results(_q):
    _params = {
        'defType': "edismax",
        'rows': "900",
        'q':"*:*"+_q+"*",
        'qf':"name"
    }
    _url = "http://solr:8983/solr/retrotech/select"# ?debugQuery=true&defType="+_deftype+"&fl"+_fl+"&fq="+_fq+"&indent=true&q.op=OR&q=*:*&qf=name&rows=900&d=2&pt="+_pt+"&sfield="+_sfield+"&spatial=true"

    data = requests.get(url = _url, params = _params).json()
    return data['response']['numFound']
    

In [40]:
MINIMUM_RESULTS_THRESHOLD = 10

agg_df = agg_df[agg_df['target'].str.len()>2]

candidates=[]
for i,v in agg_df.iterrows():
    candidate = v['target']
    print(v['target'])
    number = has_results(v['target'])
    
    if(number >= MINIMUM_RESULTS_THRESHOLD):
        print("appending "+candidate+" ,results: "+str(number))
        candidates.append(candidate)

len(candidates)

007
appending 007 ,results: 15
1 TB
1 TB external hard drive
1 tb
1 tb External hard drive
1 tb external hard drive
1 tb hard drive
10 Kicker
appending 10 Kicker ,results: 155
10 Subwoofer
10 Subwoofers
10 kicker
10 subwoofer
10 subwoofers
10.1
10.1 netbook
1000w amp
1010095
1080
1080p
1080p Projector
1080p projector
1080p tv
1089764
1109908
1126086
1162184
1196499
1196523
1196578
1196596
1196602
1196648
12 Subwoofer
12 Subwoofer box
12 Subwoofers
12 box
12 kicker
12 subwoofer
12 subwoofer box
12 subwoofers
120Hz
120Mm fan
120hz
120mm
120mm fan
1219686
1222817
1251309
127 hours
1284849
1290661
12V power supply
12v power supply
13 Assassins
13 assassins
13 ghosts
13 laptop
1342081 1342106 1342115 1342124
14 laptop
1404497 1416723 1416778 1416678
1450
1450lmt
1490LMT
1490lmt
15 Subwoofer
15 Subwoofers
15 inch tv
15 subWoofer
15 subwoofer
15 subwoofers
15 tv
15z
16 GB
16 Gb Flash Drive
appending 16 Gb Flash Drive ,results: 17
16 gb
16 gb Flash drive
16 gb SD
16 gb SD card
16 gb flash driv

appending 3D movie ,results: 15
3D movies
appending 3D movies ,results: 15
3D player
appending 3D player ,results: 15
3D players
appending 3D players ,results: 15
3D projector
appending 3D projector ,results: 15
3D projectors
appending 3D projectors ,results: 15
3D receiver
appending 3D receiver ,results: 15
3D samsung
appending 3D samsung ,results: 15
3D sony
appending 3D sony ,results: 15
3D starter kit
appending 3D starter kit ,results: 15
3D television
appending 3D television ,results: 15
3D televisions
appending 3D televisions ,results: 15
3D tv
appending 3D tv ,results: 15
3D tvS
appending 3D tvS ,results: 15
3D tvs
appending 3D tvs ,results: 15
3DS
appending 3DS ,results: 41
3DS games
appending 3DS games ,results: 39
3DTV
3Ds
3Ds case
3Dtv
3G glasses
3G router
3G tablet
3GS
3GS case
3TB
3d Blu ray
3d Blu ray player
3d Blu-Ray
3d Blu-ray
3d Blu-ray player
3d Bluray
3d DS
3d DVD
3d DVD player
3d DVDs
3d Ds
3d Glasses
3d HDTV
3d HDmi
3d Led tv
3d Lg
3d Monitors
3d Samsung
appending

AV Switch
AV adapter
AV cable
AV cable for iPhone
AV receiver
AV receivers
AVANT
AVH-P3300BT
AVH-P4300DVD
AWOLNATION
AWOLnation
Aaliyah
Aaron Lewis
Aaron lewis
Ac Adapter
Ac adapter
Ac adapters
Ac adaptor
Ac adaptors
Ac dc converter
Ac inverter
Ac power adapter
Ac power cord
Ac/DC
Ac/dc
Accer
Access point
Accessories
According to jim
Accordion
Acdc
Ace Combat
Ace Combat Assault Horizon
appending Ace Combat Assault Horizon ,results: 10
Ace Hood
appending Ace Hood ,results: 16
Ace Ventura
Ace combat
Ace combat assault horizon
Ace hood
Ace ventura
Acer
Acer 14
Acer 500
Acer A100
Acer A500
Acer Aspire
Acer Charger
Acer Iconia
Acer Iconia A500
Acer Iconia Tab
Acer Iconia Tab A500
Acer Iconia tablet
Acer Laptop
Acer Laptop Charger
Acer Laptop charger
Acer Laptops
Acer Netbook
Acer Tablet
Acer Tablet cases
Acer a100
Acer a500
Acer aspire
Acer aspire laptop
Acer aspire one
Acer aspire s3
Acer battery
Acer case
Acer charger
Acer computer
Acer desktop
Acer i5
Acer iconIa
Acer iconia
Acer iconia 

Apple chargers
Apple compUter
Apple composite
Apple composite av cable
Apple composite cable
Apple computer
Apple computerS
Apple computers
Apple desk top
Apple desktop
Apple desktop computer
Apple desktops
Apple digital AV adapter
Apple digital av adapter
Apple display
Apple dock
Apple dvi
Apple earbuds
Apple earphone
Apple earphones
Apple express
Apple hard drive
Apple hdmi
Apple headphone
Apple headphones
Apple i pad
Apple iBook
Apple iMac
Apple iPad
Apple iPad 2
appending Apple iPad 2 ,results: 169
Apple iPad 2 3G
Apple iPad case
Apple iPad tablet
Apple iPad2
Apple iPhone
Apple iPhone 4
appending Apple iPhone 4 ,results: 31
Apple iPhone 4S
Apple iPhone 4s
Apple iPhone charger
Apple iPod
Apple iPod Charger
Apple iPod Touch
Apple iPod charger
Apple iPod nano
Apple iPod touch
Apple iPods
Apple ibook
Apple imAc
Apple imac
Apple ipad
Apple ipad 2
appending Apple ipad 2 ,results: 169
Apple ipad 2 3G
Apple ipad 2 3g
Apple ipad case
Apple ipad tablet
Apple ipad2
Apple ipads
Apple iphone
Ap

Batman arkham city
Batman arkham city PS3
Batman arkham city Ps3
Batman arkham city collectors edition
Batman arkham city ps3
Batman arkum city
Batman begins
Batman beyond
Batman blu ray
Batman city
Batman dvd
Batman guide
Batman ps3
Batman the animated series
Batman year one
Batman: Arkham City
appending Batman: Arkham City ,results: 16
Batman: Year One
appending Batman: Year One ,results: 73
Batman: arkham city
Batman: year one
BattLefield 3
appending BattLefield 3 ,results: 107
Batteries
Battery
Battery Charger
Battery Pack
Battery back up
Battery backup
Battery backups
Battery case
Battery charger
Battery chargers
Battery grip
Battery laptop
Battery operated tv
Battery pack
Battle
appending Battle ,results: 29
Battle field
Battle field 3
appending Battle field 3 ,results: 107
Battle la
Battle los Angeles
Battle los angeles
BattleField
BattleStar
Battlefeild 3
appending Battlefeild 3 ,results: 107
Battlefield
Battlefield 2
appending Battlefield 2 ,results: 169
Battlefield 3
appendin

...

three musketeers
three six mafia
threes company
thrice
thrill
thrive
thrive accessories
thrive case
thrive tablet
throne
thrustmaster
thumb drive
thumb drives
thumbies
thunder bolt
thunder cats
thunderbolt
thunderbolt battery
thunderbolt case
thunderbolt cases
thunderbolt display
thundercats
thx
ti 84
ti 89
ti calculator
ti-83
ti-84
ti-89
ti84
tiger woods
tiger woods 12
appending tiger woods 12 ,results: 18
tiger woods golf
tim McGraw
tim mcgraw
timbaland
timbuk2
time capsule
time crisis
times of grace
timex
tin man
tinkerbell
titan
titanic
tivo
tivo elite
tivo remote
tivo wireless adapter
tm 103
tmnt
tmobile
tmobile cell phone
tmobile cell phones
tmobile phones
tmobile prepaid
tmobile prepaid phones
tna
to save a life
toaster
toaster oven
toaster ovens
toasters
toby keith
tochpad
toichpad
tokio hotel
tokyo drift
tom Clancy
tom and jerry
tom clancy
tom petty
tom tom
tom tom 1535
tom tom gps
tom tom xxl 550
tom waits
tom waits bad as me
tomb raider
tombstone
tomtom
tomtom GPS
tomtom gps

unlocked iphone 4
appending unlocked iphone 4 ,results: 31
unlocked mobile phones
unlocked phone
unlocked phones
unstoppable
unto the locust
up touchpad
upgrade
upright freezer
upright freezers
uprock
ups
urban cowboy
usb
usb 3.0
usb 3.0 cable
usb Bluetooth
usb Mic
appending usb Mic ,results: 173
usb WiFi
usb adapter
usb audio interface
usb bluetooth
usb cable
usb cables
usb car
usb car charger
usb card reader
usb cd drive
usb charger
usb connector
usb cord
usb cords
usb drive
usb drives
usb dvd
usb dvd drive
usb ethernet adapter
usb extender
usb extension
usb extension cable
usb flash
usb flash drive
usb flash drives
usb flashdrive
usb hard drive
usb hard drive adapter
usb hdmi
usb hdmi cable
usb headphones
usb headset
usb hub
usb hubs
usb keyboard
usb keypad
usb memory
usb memory stick
usb mic
usb microphone
usb microphones
usb mixer
usb modem
usb mouse
usb network
usb network adapter
usb optical drive
usb port
usb ports
usb router
usb sound card
usb soundcard
usb speakers
usb splitt

wilson electronics
wilson signal booster
wilson sleek
win win
window 7
appending window 7 ,results: 22
window air conditioner
window fan
windows
windows 7
appending windows 7 ,results: 22
windows 7 home
windows 7 home premium
windows 7 laptopS
windows 7 laptops
windows 7 oem
windows 7 phone
windows 7 pro
windows 7 professional
windows 7 software
windows 7 ultimate
windows 7 upgrade
windows 8
appending windows 8 ,results: 17
windows for Mac
appending windows for Mac ,results: 52
windows for mac
windows office
windows phone
windows recovery
windows seven
windows tablet
windows upgrade
windows vista
windows xp
windows7
windshield mount
wine
wine cellar
wine chiller
wine cooler
wine coolers
wine fridge
wine refrigerator
wine refrigerators
winnie
winnie the pooh
winnie the pooh blu ray
winterland
wintv
wipeout
wire
wire cover
wire covers
wire harness
wireLess surround sound
wired keyboard
wired mouse
wired router
wired xbox 360 controller
wireless
wireless Adapter
wireless DSL modem
wireles

xoom
xoom 3g
xoom accesories
xoom accessories
xoom case
xoom charger
xoom cover
xoom dock
xoom family
xoom family edition
xoom keyboard
xoom motorola
xoom screen protector
xoom tablet
xp20
xp500
xperia
xperia play
xplane
xps
xps 15
xps 15z
xps 17
xps 8300
xtrememac
y adapter
y cable
yamahA
yamaha
yamaha dock
yamaha home theater
yamaha htib
yamaha keyboard
yamaha keyboards
yamaha piano
yamaha receiver
yamaha receivers
yamaha reciever
yamaha reciver
yamaha rx
yamaha speakers
year one
yelawolf
yellowcard
yo gabba gabba
yo gotti
yoga
yoga dvd
yoke
young frankenstein
young jeezy
young jeezy tm 103
young justice
young money
young the giant
your Highness
your highness
yu yu hakusho
yugioh
yurbuds
z ro
z-ro
z-ro meth
z623
zac brown
zac brown band
zag
zagg
zagg iPhone
zagg ipad 2
appending zagg ipad 2 ,results: 169
zagg iphone
zagg iphone 4
appending zagg iphone 4 ,results: 31
zaggfolio
zboost
zeldA
zelda
zelda skyward sword
zen
zenbook
zeo
zeppelin
zeppelin air
zip drive
zodiac
zoe
zoids
zojir

2315

This "lengthy" process narrowed down our suggestions to only 2.3K, which is a significant reduction from the ~1M we had before! Now ready for indexing them into  `retrotech_signals` core.

In [71]:
import json
import random

def index_batch_to_solr(batch_data, solr_url, solr_core):
    """
    Index a batch of data to Solr.
    """
    headers = {'Content-type': 'application/json'}
    data = json.dumps(batch_data)
    response = requests.post(f'{solr_url}/{solr_core}/update?commitWithin=1000', data=data, headers=headers)
    return response

def batch_indexing(df, batch_size, solr_url, solr_core):
    """
    Index data in batches.
    """
    for i in range(0, len(df), batch_size):
        batch = df.iloc[i:i + batch_size].to_dict(orient='records')

        print(f"Indexing batch" + str(i))
        response = index_batch_to_solr(batch, solr_url, solr_core)
        if response.status_code != 200:
            print(f"Error indexing batch {i // batch_size}: {response.text}")


i_df = pd.DataFrame(candidates)
i_df.columns = ['name']

print(random.randrange(99999))

i_df['upc'] = i_df.apply(lambda _:random.randint(0, 99999999), axis=1)


batch_indexing(i_df,100,"http://solr:8983/solr","retrotech_signals")
    

30644
Indexing batch0
Indexing batch100
Indexing batch200
Indexing batch300
Indexing batch400
Indexing batch500
Indexing batch600
Indexing batch700
Indexing batch800
Indexing batch900
Indexing batch1000
Indexing batch1100
Indexing batch1200
Indexing batch1300
Indexing batch1400
Indexing batch1500
Indexing batch1600
Indexing batch1700
Indexing batch1800
Indexing batch1900
Indexing batch2000
Indexing batch2100
Indexing batch2200
Indexing batch2300


With this overly-simplistic and faulty process, we left some of the important steps for suggestion candidate-generation, though:

- We are not lowercasing suggestions before indexing, which repeats suggestions by itself
- We are not boosting before indexing any term before the other, despite we could, as we have more signals on our system (clicks, frequency vs uniqueness of queries)
- All punctuaation from terms that might not be eligible for searching has to be removed